In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.dataset import random_split
from ignite.contrib.metrics.regression import R2Score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
root = "total.csv"
batch_size = 256
epochs = 500
learning_rate = 1e-3

In [3]:
data_pd = pd.read_csv(root)
data_pd.head()

,Id,Shape,Weight,Clarity,Colour,Cut,Polish,Symmetry,Fluorescence,Length,Width,Depth,Price
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,5.05,4.35,2.94,1378.65
1,1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,4.60,4.31,2.92,1379.74
2,1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,4.71,4.35,2.94,1380.19
3,1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,4.91,4.26,2.88,1380.61
4,1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,4.70,4.46,3.01,1383.13


In [4]:
data_numpy = data_pd.to_numpy()

In [5]:
for i in [1, 3, 4, 5, 6, 7, 8]:
    wordset = {word: idx for idx, word in enumerate(np.unique(data_numpy[:,i]))}
    print(wordset)
    for row in range(len(data_numpy)):
        data_numpy[row][i] = wordset[data_numpy[row][i]]

{'CUSHION': 0, 'EMERALD': 1, 'HEART': 2, 'MARQUISE': 3, 'OVAL': 4, 'PEAR': 5, 'PRINCESS': 6, 'ROUND': 7}
{'FL': 0, 'I1': 1, 'I2': 2, 'I3': 3, 'IF': 4, 'SI1': 5, 'SI2': 6, 'VS1': 7, 'VS2': 8, 'VVS1': 9, 'VVS2': 10}
{'D': 0, 'E': 1, 'F': 2, 'FANCY': 3, 'G': 4, 'H': 5, 'I': 6, 'J': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'O': 12, 'O-P': 13, 'Q-R': 14, 'S-T': 15, 'U-V': 16, 'W': 17, 'W-X': 18, 'Y-Z': 19}
{'EX': 0, 'F': 1, 'GD': 2, 'VG': 3}
{'EX': 0, 'F': 1, 'GD': 2, 'VG': 3}
{'EX': 0, 'FR': 1, 'GD': 2, 'VG': 3}
{'F': 0, 'M': 1, 'N': 2, 'SL': 3, 'ST': 4, 'VS': 5, 'VSL': 6}


In [6]:
data_numpy = data_numpy[:,1:]
print(data_numpy[0])

[0 0.55 6 1 0 0 3 2 5.05 4.35 2.94 1378.65]


In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,ints, floats, target):
        super(Dataset).__init__()
        self.ints = ints
        self.floats = floats
        self.target = target
        
    def __getitem__(self,idx):
        return self.ints[idx],self.floats[idx], self.target[idx]
    
    def __len__(self):
        return len(self.ints)

In [8]:
data_int = torch.from_numpy(np.array(data_numpy[:,[0,2,3,4,5,6,7]], dtype="int"))
data_float = torch.from_numpy(np.array(data_numpy[:,[1,8,9,10]], dtype="float")).float()
data_target = torch.from_numpy(np.array(data_numpy[:,[11]], dtype="float")).float()
print(data_int[0])
print(data_float[0])
print(data_target[0])

tensor([0, 6, 1, 0, 0, 3, 2], dtype=torch.int32)
tensor([0.5500, 5.0500, 4.3500, 2.9400])
tensor([1378.6500])


In [9]:
train_length = int(len(data_numpy) * 0.6)
test_length = int(len(data_numpy) * 0.2)
val_length = len(data_numpy) - train_length - test_length

train_dataset = Dataset(data_int, data_float, data_target)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.emb1 = torch.nn.Embedding(8, 8)
        self.emb2 = torch.nn.Embedding(11, 11)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 4)
        self.emb5 = torch.nn.Embedding(4, 4)
        self.emb6 = torch.nn.Embedding(4, 4)
        self.emb7 = torch.nn.Embedding(7, 7)
        self.act = nn.ReLU()
        self.fc1 = nn.Linear(62, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, 1024)
        self.fc5 = nn.Linear(1024, 1)
        self.dropout = nn.Dropout()
    
    def forward(self, x, y):
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        x = self.dropout(self.act(self.fc1(x)))
        x = self.dropout(self.act(self.fc2(x)))
        x = self.dropout(self.act(self.fc3(x)))
        x = self.dropout(self.act(self.fc4(x)))
        return self.fc5(x)

In [11]:
model = Net().to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X1, X2, Y in train_loader:
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(X1, X2)
        loss = criterion(prediction, Y)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X1_val, X2_val, Y_val in val_loader:
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X1_val, X2_val)
            val_loss = criterion(val_prediction, Y_val)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 19648940.0000, val_loss: 12450322.0000
[Epoch:  2] Average loss: 15808350.0000, val_loss: 10467311.0000
[Epoch:  3] Average loss: 14378124.0000, val_loss: 8626487.0000
[Epoch:  4] Average loss: 11155004.0000, val_loss: 5135213.5000
[Epoch:  5] Average loss: 6890118.0000, val_loss: 2837916.5000
[Epoch:  6] Average loss: 2998686.5000, val_loss: 2898993.7500
[Epoch:  7] Average loss: 2044916.1250, val_loss: 4289210.0000
[Epoch:  8] Average loss: 1975046.7500, val_loss: 3350344.7500
[Epoch:  9] Average loss: 1722086.5000, val_loss: 2600471.7500
[Epoch: 10] Average loss: 1774152.2500, val_loss: 2907328.5000
[Epoch: 11] Average loss: 1286097.5000, val_loss: 3727262.2500
[Epoch: 12] Average loss: 831602.6875, val_loss: 3642766.0000
[Epoch: 13] Average loss: 1233677.5000, val_loss: 4539802.0000
[Epoch: 14] Average loss: 1118430.1250, val_loss: 4597267.5000
[Epoch: 15] Average loss: 1114300.0000, val_loss: 3472024.5000
[Epoch: 16] Average loss: 1269734.7500, val_loss: 

[Epoch: 134] Average loss: 782591.2500, val_loss: 3193943.5000
[Epoch: 135] Average loss: 747908.4375, val_loss: 4307140.5000
[Epoch: 136] Average loss: 783018.6250, val_loss: 2748856.2500
[Epoch: 137] Average loss: 724650.5000, val_loss: 3424865.0000
[Epoch: 138] Average loss: 886378.5000, val_loss: 3187832.5000
[Epoch: 139] Average loss: 672324.4375, val_loss: 4119236.5000
[Epoch: 140] Average loss: 795836.1250, val_loss: 3706960.5000
[Epoch: 141] Average loss: 626229.4375, val_loss: 4773625.0000
[Epoch: 142] Average loss: 1081380.3750, val_loss: 5238457.5000
[Epoch: 143] Average loss: 1074746.3750, val_loss: 4034761.2500
[Epoch: 144] Average loss: 759160.6250, val_loss: 3689842.0000
[Epoch: 145] Average loss: 1071510.2500, val_loss: 2751273.0000
[Epoch: 146] Average loss: 636487.7500, val_loss: 5083542.5000
[Epoch: 147] Average loss: 768541.4375, val_loss: 3192350.5000
[Epoch: 148] Average loss: 618464.9375, val_loss: 3213698.5000
[Epoch: 149] Average loss: 985739.2500, val_loss: 30

[Epoch: 266] Average loss: 648695.6875, val_loss: 4634219.0000
[Epoch: 267] Average loss: 542559.8125, val_loss: 3069109.0000
[Epoch: 268] Average loss: 765492.6250, val_loss: 2345713.5000
[Epoch: 269] Average loss: 868895.6875, val_loss: 4190102.0000
[Epoch: 270] Average loss: 681584.5625, val_loss: 3961577.5000
[Epoch: 271] Average loss: 804722.3125, val_loss: 3519001.0000
[Epoch: 272] Average loss: 715277.8750, val_loss: 2782555.0000
[Epoch: 273] Average loss: 551939.1250, val_loss: 2775979.5000
[Epoch: 274] Average loss: 877392.0000, val_loss: 3039367.7500
[Epoch: 275] Average loss: 853047.0000, val_loss: 2920815.7500
[Epoch: 276] Average loss: 729964.0000, val_loss: 3054423.0000
[Epoch: 277] Average loss: 754599.1250, val_loss: 4829829.0000
[Epoch: 278] Average loss: 990797.2500, val_loss: 4560101.5000
[Epoch: 279] Average loss: 1595339.8750, val_loss: 2847936.5000
[Epoch: 280] Average loss: 903353.5625, val_loss: 3648118.5000
[Epoch: 281] Average loss: 713669.0000, val_loss: 4170

[Epoch: 396] Average loss: 746296.8750, val_loss: 2564772.7500
[Epoch: 397] Average loss: 622010.8750, val_loss: 2780823.2500
[Epoch: 398] Average loss: 561745.3750, val_loss: 3298755.2500
[Epoch: 399] Average loss: 710900.6875, val_loss: 3462801.0000
[Epoch: 400] Average loss: 645031.5625, val_loss: 3063947.0000
[Epoch: 401] Average loss: 1009037.4375, val_loss: 4640441.0000
[Epoch: 402] Average loss: 758730.2500, val_loss: 3082844.2500
[Epoch: 403] Average loss: 713068.6875, val_loss: 3692791.2500
[Epoch: 404] Average loss: 561932.0000, val_loss: 3157515.0000
[Epoch: 405] Average loss: 842797.0000, val_loss: 2477194.5000
[Epoch: 406] Average loss: 585812.6250, val_loss: 2968327.0000
[Epoch: 407] Average loss: 721180.1250, val_loss: 3548560.2500
[Epoch: 408] Average loss: 440074.5625, val_loss: 2549800.5000
[Epoch: 409] Average loss: 527607.3125, val_loss: 3552313.0000
[Epoch: 410] Average loss: 491003.3438, val_loss: 4311402.5000
[Epoch: 411] Average loss: 630771.3750, val_loss: 4357

In [13]:
metric = R2Score(device=device)
metric.reset()

model.eval()
criterion.eval()
with torch.no_grad():
    test_avg_acc = 0
    for X1_test, X2_test, Y_test in test_loader:
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        
        test_prediction = model(X1_test, X2_test)
        metric.update([test_prediction, Y_test])
    print(f"Accuracy: {metric.compute()*100:.2f}%")

Accuracy: 96.86%
